# Generación de Coordenadas de Países

Este notebook obtiene las coordenadas (latitud y longitud) de los países en nuestro dataset usando geopy y las guarda en un archivo JSON para usar en la aplicación Streamlit.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import json
from geopy.geocoders import Nominatim
import time
import numpy as np
from pathlib import Path

In [ ]:
# Cargar datos originales
df = pd.read_csv('../reports/traits_geno_aligned.csv')
print(f"Dataset cargado con {len(df)} filas")

In [ ]:
# Configurar geocoder
geolocator = Nominatim(user_agent="rice_eda_2025")

def safe_geocode(location_name):
    """Obtiene las coordenadas de un lugar de forma segura"""
    try:
        location = geolocator.geocode(location_name, timeout=10)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error con {location_name}: {str(e)}")
    return None, None

# Obtener lista única de países
countries = df['country'].dropna().unique()
coordinates_map = {}

print("Obteniendo coordenadas de países...")
for country in countries:
    # Intentar con el nombre original
    lat, lon = safe_geocode(country)
    
    # Si falla, intentar reemplazando guiones bajos por espacios
    if lat is None:
        lat, lon = safe_geocode(country.replace('_', ' '))
    
    if lat is not None and lon is not None:
        coordinates_map[country] = [lat, lon]
        print(f"✓ {country}: {lat:.2f}°, {lon:.2f}°")
    else:
        print(f"✗ {country}: NO encontrado")
    
    # Esperar entre peticiones para no sobrecargar el servicio
    time.sleep(1.5)

print(f"\n✓ Coordenadas obtenidas para {len(coordinates_map)} países")

In [ ]:
# Fallback manual para países que puedan fallar
fallback_coordinates = {
    'South_Korea': [35.9078, 127.7669],
    'North_Korea': [40.3399, 127.5101],
    'Taiwan': [23.5937, 121.0254]
}

# Añadir fallbacks al mapa de coordenadas
coordinates_map.update(fallback_coordinates)

# Guardar el mapa de coordenadas en un archivo JSON
output_path = Path('../data/country_coordinates.json')
output_path.parent.mkdir(exist_ok=True)

with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(coordinates_map, f, indent=4, ensure_ascii=False)

print(f"✓ Coordenadas guardadas en {output_path}")

In [ ]:
# Verificar el contenido del archivo
with open(output_path) as f:
    saved_coords = json.load(f)

print("Muestra de coordenadas guardadas:")
for country, coords in list(saved_coords.items())[:5]:
    print(f"{country}: {coords}")